In [1]:
import torch.nn as nn

In [9]:
class VGG(nn.Module):

  def __init__(self,features, num_classes = 1000, init_weights=True):
    #nn.Module.init()
    super(VGG,self).__init__()
    self.features = features
    self.avgpool = nn.AdaptiveAvgPool2d(7)
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(True),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Linear(4096, num_classes),
    )

    #잔여 weight 제거
    if init_weights:
      self._initialize_weights()

  def forward(self,x):
    x = self.features(x)
    x = self.avgpool(x)
    x = x.view(x.size(0),-1)
    x = self.classifier(x)
    return x

  def _initialize_weights(self):
    #self.modules 모델 클래스의 layer들
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        #kaiming_normal_ == he_normal 
        #std == gain / (fan_mode) ** (1/2)
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
          #m.bias 를 0 으로 초기화
          nn.init.constant_(m.bias,0)
      elif isinstance(m,nn.BatchNorm2d):
        nn.init.constant_(m.weight,1)
        nn.init.constant_(m.bias,0)
      
      elif isinstance(m,nn.Linear):
        #m.weight tensor를 mean(0), std(0.01)로 초기화
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias,0)
      
#vgg 모델에 따라 layer 개수 다르게 생성
def make_layers(cfg, batch_norm=False):
  layers=[]
  in_channels = 3
  for v in cfg:
    if v == 'M':
      layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
    
    else :
      conv2d = nn.Conv2d(in_channels,v,kernel_size=3, padding=1)
      if batch_norm:
        layers+= [conv2d, nn.BatchNorm2d(v),nn.ReLU(inplace=True)]
      else:
        layers += [conv2d, nn.ReLU(inplace=True)]
      #output 이었던 v가 input이 됨
      in_channels = v
  
  return nn.Sequential(*layers)

#model 구조 
#'M' maxpool / num filter 수  
cfgs = {
  #vgg11 == 8 + self.classifier 3  == 11
  'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
  #vgg13
  'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
  #vgg16
  'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
  #vgg 19
  'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [12]:
feature = make_layers(cfgs['D'], batch_norm = True)
CNN = VGG (feature, num_classes=10, init_weights= True)

In [13]:
CNN

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256